In [1]:
import sys; sys.path.insert(0, '..')
from tensorflow.keras.metrics import CategoricalAccuracy
import tensorflow as tf
from data_loader import get_loader
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from models.distances import Euclidean_Distance
from models.stn import BilinearInterpolation,Localization
from sklearn.neighbors import KNeighborsClassifier
from time import time

In [2]:
encoder = keras.models.load_model(
    '../model_files/best_encoders/densenet_gtsrb2tt100k_encoder.h5',
    custom_objects={'BilinearInterpolation':BilinearInterpolation,'Localization':Localization},compile=False)

In [3]:
loader = get_loader('gtsrb2tt100k') 
test_generator = loader.get_test_generator(batch=8,dim=64)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\MTPNet\\ipynb\\datasets/TT100K\\template\\all'

In [6]:
t = iter(test_generator)
[Xs,Xq],y = next(t)
Zs = encoder(Xs)

In [7]:
knn = KNeighborsClassifier(n_neighbors=1,n_jobs=-1,metric='l2')

In [8]:
acc = 0
start = time()
y_train = to_categorical(np.arange(36),num_classes=36)
knn.fit(Zs,y_train);
pb = tf.keras.utils.Progbar(len(test_generator),verbose=1)
for i,z in enumerate(test_generator):
    [Xs,Xq],y_test = z
    Zq = encoder(Xq)
    acc += knn.score(Zq,y_test)
    values=[('test acc',(acc / (i+1)))]
    pb.add(1,values=values)
end = time() - start

1498/1498 [==============================] - 184s 122ms/step - test acc: 0.8068


In [9]:
time_per_image = end / (len(test_generator) * 8)
fps = int(1/time_per_image)
mean_accuracy = np.round((acc / len(test_generator))*100,2)
print(f'One image KNN inference time is {time_per_image:.4f}s')
print(f'FPS is {fps}')
print(f'KNN accuracy is {mean_accuracy}')

One image KNN inference time is 0.0153s
FPS is 65
KNN accuracy is 80.82
